# Anomaly (supervised) Detection

In [1]:
%pip install pandas numpy matplotlib scikit-learn catboost catboost-widget

ERROR: Could not find a version that satisfies the requirement catboost-widget (from versions: none)
ERROR: No matching distribution found for catboost-widget
Note: you may need to restart the kernel to use updated packages.


# Load Data

In [2]:
import pandas as pd
df = pd.read_csv('ai4i2020.csv')

In [3]:
df.columns

Index(['UDI', 'Product ID', 'Type', 'Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]', 'Machine failure', 'TWF', 'HDF', 'PWF', 'OSF',
       'RNF'],
      dtype='object')

In [4]:
cat_features=['Product ID', 'Type']
X_features= cat_features +  ['Air temperature [K]',
       'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]',
       'Tool wear [min]']
y_features=[ 'TWF', 'HDF', 'PWF', 'OSF', 'RNF']

In [5]:
X=df[X_features]
y=df[y_features]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# EDA

In [7]:
df['Machine failure'].value_counts()

Machine failure
0    9661
1     339
Name: count, dtype: int64

In [8]:
df[y_features].value_counts()

TWF  HDF  PWF  OSF  RNF
0    0    0    0    0      9652
     1    0    0    0       106
     0    1    0    0        80
          0    1    0        78
1    0    0    0    0        42
0    0    0    0    1        18
          1    1    0        11
     1    0    1    0         6
          1    0    0         3
1    0    0    1    0         2
               0    1         1
          1    1    0         1
Name: count, dtype: int64

# Model

In [15]:
from catboost import CatBoostClassifier
from sklearn import metrics
import numpy as np
def train_and_test(on, X_train, y_train, X_val, y_val, cat_features,iterations=800):

    _y_val = y_val[on].values.flatten()
    _y_train = y_train[on].values.flatten()
    
    clf = CatBoostClassifier(
        iterations=iterations,
        random_seed=42,
        loss_function='Logloss',
        cat_features=cat_features,
        auto_class_weights='Balanced'
    )
    
    clf.fit(
        X_train, _y_train,
        verbose=False
    )
    
    y_pred = clf.predict(X_val)
    print("Report for:", on)
    print(metrics.accuracy_score(_y_val, y_pred))
    print(metrics.balanced_accuracy_score(_y_val, y_pred))
    print(metrics.confusion_matrix(_y_val, y_pred))

    feature_importance = clf.feature_importances_
    sorted_idx = np.argsort(-1*feature_importance)
    for col,imp in zip(X_train.columns[sorted_idx], feature_importance[sorted_idx]):
        print(f"{col} is {int(imp)}% important on {on}")

for on in y_features:
    train_and_test(on, X_train, y_train, X_test, y_test, cat_features)

Report for: TWF
0.986
0.5452261306532663
[[1971   19]
 [   9    1]]
Tool wear [min] is 42% important on TWF
Torque [Nm] is 16% important on TWF
Air temperature [K] is 14% important on TWF
Rotational speed [rpm] is 11% important on TWF
Process temperature [K] is 8% important on TWF
Type is 6% important on TWF
Product ID is 0% important on TWF
Report for: HDF
0.995
0.9974721941354904
[[1968   10]
 [   0   22]]
Rotational speed [rpm] is 37% important on HDF
Air temperature [K] is 33% important on HDF
Process temperature [K] is 16% important on HDF
Torque [Nm] is 6% important on HDF
Tool wear [min] is 2% important on HDF
Type is 2% important on HDF
Product ID is 0% important on HDF
Report for: PWF
0.994
0.941921740105393
[[1972   10]
 [   2   16]]
Torque [Nm] is 47% important on PWF
Rotational speed [rpm] is 34% important on PWF
Type is 7% important on PWF
Tool wear [min] is 5% important on PWF
Air temperature [K] is 3% important on PWF
Process temperature [K] is 2% important on PWF
Produc